In [8]:
pip install langdetect

In [9]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime, timedelta
from langdetect import detect

root = "https://www.google.com/"

# Initial link for the first page
link_base = "https://www.google.com/search?q=tesla&rlz=1C1GCEA_enMA1047MA1047&tbas=0&tbs=qdr:d,sbd:1&tbm=nws&source=lnt&sa=X&ved=2ahUKEwiAmILcsM2AAxWyd6QEHbwdARUQpwV6BAgCEBM&biw=1366&bih=651&dpr=1"
articles_per_page = 10
num_pages = 25

data = []

# Loop through multiple pages
for page in range(num_pages):
    start_index = page * articles_per_page
    link = f"{link_base}&start={start_index}"

    req = Request(link, headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'})
    webpage = urlopen(req).read()

    with requests.Session() as c:
        soup = BeautifulSoup(webpage, 'html.parser')
        articles = soup.find_all('a', class_='WlydOe')

    for article in articles:
        time_element = article.find('div', class_='OSrXXb')
        title_element = article.find('div', class_='n0jPhd')
        description_element = article.find('div', class_='GI74Re')

        if time_element:
            time_string = time_element.span.get_text()
            if "min" in time_string:
                minutes_ago = int(time_string.split()[0])
                time = datetime.now() - timedelta(minutes=minutes_ago)
            elif "hour" in time_string:
                hours_ago = int(time_string.split()[0])
                time = datetime.now() - timedelta(hours=hours_ago)
            else:
                time = "N/A"
        else:
            time = "N/A"

        # Convert time to date and time strings
        date_str = time.strftime("%Y-%m-%d")
        time_str = time.strftime("%H:%M:%S")

        if title_element:
            title = title_element.get_text()
        else:
            title = "N/A"

        if description_element:
            description = description_element.get_text()
        else:
            description = "N/A"

        description_language = detect(description)
        if description_language == 'en':
            data.append({'Date': date_str, 'Time': time_str, 'Title': title, 'Description': description})

df = pd.DataFrame(data)

output_file = 'tesla_articles13.xlsx'
df.to_excel(output_file, index=False)
print(f"Data has been scraped and saved to {output_file}.")


Data has been scraped and saved to tesla_articles13.xlsx.
